# 🦷 Teeth Segmentation Training - 16 Classes

Train a U-Net model for individual tooth segmentation using Roboflow COCO dataset.

**Dataset:** `segmentation-dental` with 16 tooth classes

**Classes:** teeth, teeth_2, teeth_3, teeth_4, teeth_5, teeth_6, teeth_7, teeth_8, teeth_9, teeth_10, teeth_11, teeth_12, teeth_13, teeth_14, teeth_15, teeth_16

---

## 📋 Step 1: Check GPU

Make sure GPU is enabled: **Runtime → Change runtime type → Hardware accelerator → GPU (T4)**

In [ ]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ WARNING: GPU not available. Training will be slow!")

## 📦 Step 2: Install Dependencies

In [ ]:
!pip install -q roboflow pycocotools pillow matplotlib seaborn scikit-learn
print("✅ All packages installed successfully!")

## 📥 Step 3: Download Dataset from Roboflow

In [ ]:
from roboflow import Roboflow

rf = Roboflow(api_key="O0qh97MHEruEAi4iUdAl")
project = rf.workspace("mv-dsnw8").project("segmentation-dental")
version = project.version(1)
dataset = version.download("coco-segmentation")

print(f"\n✅ Dataset downloaded to: {dataset.location}")

## 🏗️ Step 4: Define U-Net Model

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2), DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.up = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True)
        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]
        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2, diffY // 2, diffY - diffY // 2])
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class UNet(nn.Module):
    def __init__(self, num_classes=16, in_channels=3):
        super().__init__()
        # Encoder
        self.inc = DoubleConv(in_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        # Bottleneck
        self.bottom = Down(512, 1024)
        # Decoder
        self.up1 = Up(1024 + 512, 512)
        self.up2 = Up(512 + 256, 256)
        self.up3 = Up(256 + 128, 128)
        self.up4 = Up(128 + 64, 64)
        self.outc = nn.Conv2d(64, num_classes, kernel_size=1)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.bottom(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

print("✅ U-Net model defined")

## 🗂️ Step 5: Create Dataset Class

In [ ]:
import os
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from pycocotools.coco import COCO

class CocoSegmentationDataset(Dataset):
    def __init__(self, img_dir, ann_file, img_size=256, transforms=None):
        super().__init__()
        self.img_dir = img_dir
        self.img_size = img_size
        self.coco = COCO(ann_file)
        self.img_ids = list(self.coco.imgs.keys())
        self.transforms = transforms
        
        # 16 tooth classes
        self.class_name_to_idx = {
            "teeth": 0, "teeth_2": 1, "teeth_3": 2, "teeth_4": 3,
            "teeth_5": 4, "teeth_6": 5, "teeth_7": 6, "teeth_8": 7,
            "teeth_9": 8, "teeth_10": 9, "teeth_11": 10, "teeth_12": 11,
            "teeth_13": 12, "teeth_14": 13, "teeth_15": 14, "teeth_16": 15,
        }
        
        # Build mapping from COCO category_id -> class index
        self.cat_id_to_class_idx = {}
        for cat in self.coco.loadCats(self.coco.getCatIds()):
            name = cat["name"]
            if name in self.class_name_to_idx:
                self.cat_id_to_class_idx[cat["id"]] = self.class_name_to_idx[name]
    
    def __len__(self):
        return len(self.img_ids)
    
    def __getitem__(self, idx):
        img_id = self.img_ids[idx]
        img_info = self.coco.loadImgs(img_id)[0]
        img_path = os.path.join(self.img_dir, img_info["file_name"])
        
        # Load image
        img = Image.open(img_path).convert("RGB")
        
        # Create empty mask
        w, h = img_info["width"], img_info["height"]
        mask = np.zeros((h, w), dtype=np.uint8)
        
        # Get annotations
        ann_ids = self.coco.getAnnIds(imgIds=img_id)
        anns = self.coco.loadAnns(ann_ids)
        
        # Draw masks
        for ann in anns:
            cat_id = ann["category_id"]
            if cat_id not in self.cat_id_to_class_idx:
                continue
            class_idx = self.cat_id_to_class_idx[cat_id]
            m = self.coco.annToMask(ann)
            mask[m == 1] = class_idx
        
        # Apply transforms
        if self.transforms is not None:
            img = self.transforms(img)
        
        # Resize mask
        mask = Image.fromarray(mask)
        mask = mask.resize((self.img_size, self.img_size), resample=Image.NEAREST)
        mask = np.array(mask, dtype=np.int64)
        mask = torch.from_numpy(mask).long()
        
        return img, mask

print("✅ Dataset class defined")

## ⚙️ Step 6: Training Configuration

In [ ]:
# ========== HYPERPARAMETERS ==========
EPOCHS = 80          # Increased from 30 for better convergence
BATCH_SIZE = 8       # Good balance for GPU memory
LEARNING_RATE = 5e-4 # Slightly lower for stable training
IMG_SIZE = 256
NUM_CLASSES = 16

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {DEVICE}')


## 📊 Step 7: Create Data Loaders

In [ ]:
# Image transforms WITH DATA AUGMENTATION
from torchvision import transforms as T

# Training transforms (with augmentation)
train_transform = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.RandomHorizontalFlip(p=0.5),  # Flip teeth horizontally
    T.RandomVerticalFlip(p=0.3),    # Occasional vertical flip
    T.RandomRotation(degrees=15),   # Slight rotation
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Validation transforms (no augmentation)
val_transform = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create datasets
train_dataset = CocoSegmentationDataset(
    TRAIN_IMG_DIR, TRAIN_JSON, img_size=IMG_SIZE, transforms=train_transform
)
val_dataset = CocoSegmentationDataset(
    VAL_IMG_DIR, VAL_JSON, img_size=IMG_SIZE, transforms=val_transform
)

# Create data loaders
train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2
)
val_loader = DataLoader(
    val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2
)

print(f'✅ Data loaders created with augmentation')
print(f'  Training samples: {len(train_dataset)}')
print(f'  Validation samples: {len(val_dataset)}')
print(f'  Training batches: {len(train_loader)}')
print(f'  Validation batches: {len(val_loader)}')
print(f'\n📊 Data Augmentation enabled:')
print(f'  - Random horizontal flip (50%)')
print(f'  - Random vertical flip (30%)')
print(f'  - Random rotation (±15°)')
print(f'  - Color jitter (brightness/contrast)')


## 👁️ Step 8: Visualize Sample (Optional)

In [ ]:
import matplotlib.pyplot as plt

# Get a sample batch
sample_img, sample_mask = next(iter(train_loader))

# Visualize first image
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Denormalize image
mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
img_display = sample_img[0] * std + mean
img_display = torch.clamp(img_display, 0, 1)

axes[0].imshow(img_display.permute(1, 2, 0))
axes[0].set_title("Input Image")
axes[0].axis('off')

axes[1].imshow(sample_mask[0], cmap='tab20')
axes[1].set_title("Ground Truth Mask")
axes[1].axis('off')

plt.tight_layout()
plt.show()

print(f"Image shape: {sample_img.shape}")
print(f"Mask shape: {sample_mask.shape}")
print(f"Unique classes: {torch.unique(sample_mask)}")

## 🚀 Step 9: Training Loop

In [ ]:
# Initialize model
model = UNet(num_classes=NUM_CLASSES, in_channels=3).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss()

print(f"✅ Model initialized with {sum(p.numel() for p in model.parameters())/1e6:.2f}M parameters")
print(f"\n{'='*70}")
print(f"🔥 Starting Training...")
print(f"{'='*70}\n")

best_val_loss = float("inf")
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(EPOCHS):
    # TRAINING
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    
    for images, masks in train_loader:
        images = images.to(DEVICE)
        masks = masks.to(DEVICE)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * images.size(0)
        
        # Accuracy
        pred = torch.argmax(outputs, dim=1)
        train_correct += (pred == masks).sum().item()
        train_total += masks.numel()
    
    train_loss /= len(train_loader.dataset)
    train_acc = 100.0 * train_correct / train_total
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    
    # VALIDATION
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for images, masks in val_loader:
            images = images.to(DEVICE)
            masks = masks.to(DEVICE)
            outputs = model(images)
            loss = criterion(outputs, masks)
            val_loss += loss.item() * images.size(0)
            
            # Accuracy
            pred = torch.argmax(outputs, dim=1)
            val_correct += (pred == masks).sum().item()
            val_total += masks.numel()
    
    val_loss /= len(val_loader.dataset)
    val_acc = 100.0 * val_correct / val_total
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)
    
    # Print progress
    print(f"Epoch [{epoch+1:2d}/{EPOCHS}] | "
          f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | "
          f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%", end="")
    
    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
        print(" ✓ Best!")
    else:
        print()

print(f"\n{'='*70}")
print(f"✅ Training Complete!")
print(f"Best Validation Loss: {best_val_loss:.4f}")
print(f"Model saved to: {MODEL_SAVE_PATH}")
print(f"{'='*70}")

## 📈 Step 10: Plot Training History

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Loss plot
axes[0].plot(range(1, EPOCHS + 1), train_losses, label='Train Loss', marker='o', linewidth=2)
axes[0].plot(range(1, EPOCHS + 1), val_losses, label='Val Loss', marker='s', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Accuracy plot
axes[1].plot(range(1, EPOCHS + 1), train_accuracies, label='Train Accuracy', 
             marker='o', linewidth=2, color='green')
axes[1].plot(range(1, EPOCHS + 1), val_accuracies, label='Val Accuracy', 
             marker='s', linewidth=2, color='orange')
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Accuracy (%)', fontsize=12)
axes[1].set_title('Training and Validation Accuracy', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 📊 Step 11: Confusion Matrix

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Load best model
model.load_state_dict(torch.load(MODEL_SAVE_PATH))
model.eval()

# Collect predictions
all_preds = []
all_targets = []

print("🔄 Generating predictions...")
with torch.no_grad():
    for images, masks in val_loader:
        images = images.to(DEVICE)
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1)
        
        all_preds.append(preds.cpu().numpy().flatten())
        all_targets.append(masks.numpy().flatten())

all_preds = np.concatenate(all_preds)
all_targets = np.concatenate(all_targets)

# Compute confusion matrix
cm = confusion_matrix(all_targets, all_preds, labels=range(NUM_CLASSES))
cm_normalized = cm.astype('float') / (cm.sum(axis=1)[:, np.newaxis] + 1e-10)

# Class names
class_names = ['teeth', 'teeth_2', 'teeth_3', 'teeth_4', 'teeth_5', 'teeth_6',
               'teeth_7', 'teeth_8', 'teeth_9', 'teeth_10', 'teeth_11', 'teeth_12',
               'teeth_13', 'teeth_14', 'teeth_15', 'teeth_16']

# Plot
plt.figure(figsize=(14, 12))
sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='RdYlBu_r', 
            xticklabels=class_names, yticklabels=class_names,
            cbar_kws={'label': 'Normalized Count'},
            vmin=0, vmax=1, linewidths=0.5, linecolor='gray')

plt.title('Confusion Matrix - Per Class Normalized', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Predicted Class', fontsize=13, fontweight='bold')
plt.ylabel('True Class', fontsize=13, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Per-class accuracy
print("\n📊 Per-Class Accuracy:")
print("=" * 40)
for i, name in enumerate(class_names):
    acc = cm_normalized[i, i] * 100
    print(f"{name:12s}: {acc:6.2f}%")
print("=" * 40)
print(f"Mean Accuracy: {np.mean(np.diag(cm_normalized)) * 100:.2f}%")

## 🔍 Step 12: Test Prediction Visualization

In [ ]:
# Get validation sample
val_img, val_mask = next(iter(val_loader))
val_img = val_img.to(DEVICE)

# Predict
with torch.no_grad():
    pred = model(val_img)
    pred_mask = torch.argmax(pred, dim=1)

# Visualize
idx = 0
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Denormalize
mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1).to(DEVICE)
std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1).to(DEVICE)
img_display = val_img[idx] * std + mean
img_display = torch.clamp(img_display, 0, 1).cpu()

axes[0].imshow(img_display.permute(1, 2, 0))
axes[0].set_title("Input Image")
axes[0].axis('off')

axes[1].imshow(val_mask[idx].cpu(), cmap='tab20', vmin=0, vmax=NUM_CLASSES-1)
axes[1].set_title("Ground Truth")
axes[1].axis('off')

axes[2].imshow(pred_mask[idx].cpu(), cmap='tab20', vmin=0, vmax=NUM_CLASSES-1)
axes[2].set_title("Prediction")
axes[2].axis('off')

plt.tight_layout()
plt.show()

print("✅ Prediction complete!")

## 💾 Step 13: Download Model

Download the trained model to your computer.

In [ ]:
from google.colab import files

files.download(MODEL_SAVE_PATH)
print(f"✅ Model '{MODEL_SAVE_PATH}' downloaded!")

## 🎉 Training Complete!

### Next Steps:

1. **Download the model** using Step 13
2. **Use in your project**:
   ```python
   model = UNet(num_classes=16, in_channels=3)
   model.load_state_dict(torch.load('best_unet_16class.pt'))
   model.eval()
   ```

### Model Summary:
- **Dataset**: segmentation-dental (16 tooth classes)
- **Architecture**: U-Net
- **Input Size**: 256×256
- **Output**: 16-channel segmentation mask

### Tips:
- Adjust `EPOCHS`, `BATCH_SIZE`, or `LR` for better results
- Monitor the loss/accuracy plots for overfitting
- Check confusion matrix to see which classes need improvement
- Consider data augmentation if accuracy is low